<a href="https://colab.research.google.com/github/springboardmentor0327/Text_Summarization_Infosys_Internship_Oct2024/blob/Sameer/bart_with_rouge_and_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FINETUNING OF BART with CNN-Dailymail Dataset**

In [1]:
!pip install datasets
!pip install transformers
!pip install pytorch_lightning
!pip install torch
!pip install scikit-learn
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 17.2 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
dataset_ = load_dataset('cnn_dailymail', '3.0.0', split='train[:15]')

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
print(dataset_)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 15
})


In [6]:
from pprint import pprint

print(f"👉Dataset len(dataset): {len(dataset_)}")
print("\n👉First item 'dataset[0]':")
pprint(dataset_[0])

👉Dataset len(dataset): 15

👉First item 'dataset[0]':
{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe '
            'gains access to a reported £20 million ($41.1 million) fortune as '
            "he turns 18 on Monday, but he insists the money won't cast a "
            'spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter '
            'and the Order of the Phoenix" To the disappointment of gossip '
            'columnists around the world, the young actor says he has no plans '
            'to fritter his cash away on fast cars, drink and celebrity '
            'parties. "I don\'t plan to be one of those people who, as soon as '
            'they turn 18, suddenly buy themselves a massive sports car '
            'collection or something similar," he told an Australian '
            'interviewer earlier this month. "I don\'t think I\'ll be '
            'particularly extravagant. "The things I like buying are things '
            'that cost a

In [7]:
# Importing librareis
import torch
from torch.nn import functional as F
from torch import nn
import pytorch_lightning as pl

from transformers import BartForConditionalGeneration, BartTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
from torch.utils.data import DataLoader


In [8]:
import torch
import pandas as pd
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class Dataset(torch.utils.data.Dataset):
    """Custom dataset class for text summarization using PyTorch DataLoader.

    For more information about Dataset and DataLoader, see:
    https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    """

    def __init__(self, texts, summaries, tokenizer, source_len, summ_len):
        """
        Initialize the Dataset.

        Args:
            texts (list): List of input texts.
            summaries (list): List of target summaries.
            tokenizer: Tokenizer for text encoding.
            source_len (int): Maximum length for input text.
            summ_len (int): Maximum length for target summary.
        """
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.summ_len = summ_len

    def __len__(self):
        """
        Get the number of samples in the dataset.
        """
        return len(self.summaries) - 1

    def __getitem__(self, index):
        """
        Get a single data sample from the dataset.

        Args:
            index (int): Index of the data sample to retrieve.

        Returns:
            Tuple containing:
            - source input IDs
            - source attention mask
            - target input IDs
            - target attention mask
        """
        text = ' '.join(str(self.texts[index]).split())
        summary = ' '.join(str(self.summaries[index]).split())

        # Article text pre-processing
        source = self.tokenizer.batch_encode_plus([text],
                                                  max_length=self.source_len,
                                                  pad_to_max_length=True,
                                                  return_tensors='pt')
        # Summary Target pre-processing
        target = self.tokenizer.batch_encode_plus([summary],
                                                  max_length=self.summ_len,
                                                  pad_to_max_length=True,
                                                  return_tensors='pt')

        return (
            source['input_ids'].squeeze(),
            source['attention_mask'].squeeze(),
            target['input_ids'].squeeze(),
            target['attention_mask'].squeeze()
        )

class BARTDataLoader(pl.LightningDataModule):
    '''Pytorch Lightning Model Dataloader class for BART'''

    def __init__(self, tokenizer, text_len, summarized_len, file_path,
                 corpus_size, columns_name, train_split_size, batch_size):
        """
        Initialize the BARTDataLoader.

        Args:
            tokenizer: Tokenizer for text encoding.
            text_len (int): Maximum length for input text.
            summarized_len (int): Maximum length for target summary.
            file_path (str): Path to the CSV data file.
            corpus_size (int): Number of rows to read from the CSV file.
            columns_name (list): List of column names to use.
            train_split_size (float): Size of the training split (e.g., 0.8 for 80%).
            batch_size (int): Batch size for data loading.
        """
        super().__init__()
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.summarized_len = summarized_len
        self.input_text_length = summarized_len
        self.file_path = file_path
        self.nrows = corpus_size
        self.columns = columns_name
        self.train_split_size = train_split_size
        self.batch_size = batch_size

    def prepare_data(self):
        """
        Load and preprocess the data from the CSV file.
        """
        data = pd.read_csv(self.file_path, nrows=self.nrows, encoding='latin-1')
        data = data[self.columns]
        data.iloc[:, 1] = 'summarize: ' + data.iloc[:, 1]
        self.text = list(data.iloc[:, 0].values)
        self.summary = list(data.iloc[:, 1].values)

    def setup(self, stage=None):
        """
        Split the data into training and validation sets.

        Args:
            stage (str): The current stage ('fit' or 'test').
        """
        X_train, y_train, X_val, y_val = train_test_split(
            self.text, self.summary, train_size=self.train_split_size
        )

        self.train_dataset = (X_train, y_train)
        self.val_dataset = (X_val, y_val)

    def train_dataloader(self):
        """
        Create a DataLoader for the training dataset.
        """
        train_data = Dataset(texts=self.train_dataset[0],
                             summaries=self.train_dataset[1],
                             tokenizer=self.tokenizer,
                             source_len=self.text_len,
                             summ_len=self.summarized_len)
        return DataLoader(train_data, self.batch_size)

    def val_dataloader(self):
        """
        Create a DataLoader for the validation dataset.
        """
        val_dataset = Dataset(texts=self.val_dataset[0],
                              summaries=self.val_dataset[1],
                              tokenizer=self.tokenizer,
                              source_len=self.text_len,
                              summ_len=self.summarized_len)
        return DataLoader(val_dataset, self.batch_size)


In [9]:
import torch
import pytorch_lightning as pl
from transformers import AdamW

class AbstractiveSummarizationBARTFineTuning(pl.LightningModule):
    """Abstractive summarization model class for fine-tuning BART."""
    def __init__(self, model, tokenizer):
        """
        Initialize the AbstractiveSummarizationBARTFineTuning model.

        Args:
            model: Pre-trained BART model.
            tokenizer: BART tokenizer.
        """
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.training_step_outputs = []  # Initialize to store training step outputs

    def forward(self, input_ids, attention_mask, decoder_input_ids,
                decoder_attention_mask=None, lm_labels=None):
        """
        Forward pass for the model.

        Args:
            input_ids: Input token IDs.
            attention_mask: Attention mask for input.
            decoder_input_ids: Target token IDs.
            decoder_attention_mask: Attention mask for target.
            lm_labels: Language modeling labels.

        Returns:
            Model outputs.
        """
        outputs = self.model.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            labels=decoder_input_ids
        )

        return outputs

    def preprocess_batch(self, batch):
        """
        Reformat and preprocess the batch for model input.

        Args:
            batch: Batch of data.

        Returns:
            Formatted input and target data.
        """
        input_ids, source_attention_mask, decoder_input_ids, \
        decoder_attention_mask = batch

        y = decoder_input_ids
        decoder_ids = decoder_input_ids
        source_ids = input_ids
        source_mask = source_attention_mask

        return source_ids, source_mask, decoder_ids, decoder_attention_mask, decoder_attention_mask

    def training_step(self, batch, batch_idx):
        """
        Training step for the model.

        Args:
            batch: Batch of training data.
            batch_idx: Index of the batch.

        Returns:
            Loss for the training step.
        """
        input_ids, source_attention_mask, decoder_input_ids, \
        decoder_attention_mask, lm_labels = self.preprocess_batch(batch)

        outputs = self.forward(input_ids=input_ids, attention_mask=source_attention_mask,
                               decoder_input_ids=decoder_input_ids,
                               decoder_attention_mask=decoder_attention_mask,
                               lm_labels=lm_labels
                       )
        loss = outputs.loss
        # self.training_step_outputs.append(loss.detach())  # Store loss for epoch-end calculation
        return {'loss': loss}

        return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step for the model.

        Args:
            batch: Batch of validation data.
            batch_idx: Index of the batch.

        Returns:
            Loss for the validation step.
        """
        input_ids, source_attention_mask, decoder_input_ids, \
        decoder_attention_mask, lm_labels = self.preprocess_batch(batch)

        outputs = self.forward(input_ids=input_ids, attention_mask=source_attention_mask,
                               decoder_input_ids=decoder_input_ids,
                               decoder_attention_mask=decoder_attention_mask,
                               lm_labels=lm_labels
                       )
        loss = outputs.loss

        return loss
    def on_training_epoch_end(self):
        """
        Calculate and log the average training loss for the epoch.

        Args:
            outputs: List of training step outputs.
        """
        # avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        avg_loss = torch.stack([x['loss'] for x in self.trainer.callback_metrics['loss']]).mean()
        self.log('Epoch', self.trainer.current_epoch)
        self.log('avg_epoch_loss', {'train': avg_loss})
        self.trainer.callback_metrics['loss'] = []


        # self.log('Epoch', self.trainer.current_epoch)
        # self.log('avg_epoch_loss', {'train': avg_loss})
        self.training_step_outputs.clear()  # Clear the list for the next epoch

    def val_epoch_end(self, loss):
        """
        Calculate and log the average validation loss for the epoch.

        Args:
            loss: List of validation step losses.
        """
        avg_loss = torch.stack([x["loss"] for x in loss]).mean()
        self.log('avg_epoch_loss', {'Val': avg_loss})

    def configure_optimizers(self):
        """
        Configure and return the optimizer for the model.

        Returns:
            Optimizer for training.
        """
        model = self.model
        optimizer = AdamW(model.parameters())
        self.opt = optimizer

        return [optimizer]


In [11]:
#DOWNLOAD and import curated dataset in google colab files

# Tokenizer
# Upload the curated_data_subset.csv if using Colab or change the path to a local file
model_ = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Dataloader
# Initialize a DataLoader for processing and loading data
dataloader = BARTDataLoader(tokenizer=tokenizer, text_len=512,
                            summarized_len=150,
                            file_path='curated_data.csv',
                            corpus_size=50, columns_name=['article_content','summary'],
                            train_split_size=0.8, batch_size=2)

# Read and pre-process data
dataloader.prepare_data()

# Train-test Split
# Split the data into training and validation sets
dataloader.setup()



# Main Model class
# Create an instance of the AbstractiveSummarizationBARTFineTuning model
model = AbstractiveSummarizationBARTFineTuning(model=model_, tokenizer=tokenizer)




# Trainer Class
# Initialize a PyTorch Lightning Trainer for training and evaluation
# Specify the accelerator and devices instead of gpus
# Check if a GPU is available before setting the accelerator
# If no GPU is available, default to CPU
import torch
if torch.cuda.is_available():
    trainer = pl.Trainer(check_val_every_n_epoch=1, max_epochs=5, accelerator="gpu", devices=1)
else:
    trainer = pl.Trainer(check_val_every_n_epoch=1, max_epochs=5, accelerator="cpu")

# Fit model
# Train the model using the specified trainer and data loader
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params | Mode
--------------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M  | eval
--------------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)
0         Modules in train mode
1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of trai

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [12]:
# Main Model class
# Create an instance of the AbstractiveSummarizationBARTFineTuning model
model = AbstractiveSummarizationBARTFineTuning(model=model_, tokenizer=tokenizer)

In [13]:
import torch
import pytorch_lightning as pl
from transformers import AdamW, BartForConditionalGeneration, BartTokenizer
if torch.cuda.is_available():
    trainer = pl.Trainer(check_val_every_n_epoch=1, max_epochs=5, accelerator="gpu", devices=1)
else:
    trainer = pl.Trainer(check_val_every_n_epoch=1, max_epochs=5, accelerator="cpu")

# Trainer Class
# Initialize a PyTorch Lightning Trainer for training and evaluation
# You can specify the number of GPUs (e.g., gpus=1) if available, or remove it if not.
# trainer = pl.Trainer(check_val_every_n_epoch=1, max_epochs=5, gpus=1)

# Fit model
# Train the model using the specified trainer and data loader
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params | Mode
--------------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M  | eval
--------------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)
0         Modules in train mode
1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [14]:
from transformers import BartTokenizer, BartForConditionalGeneration

def summarize_article(article, max_words=50):
    # Load BART model and tokenizer
    model_name = 'facebook/bart-large-cnn'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Define minimum and maximum words for the summary
    min_words = max(10, max_words - 10)
    max_tokens = max_words * 1.5  # Adding a buffer as BERT-based tokens don't align 1:1 with words

    # Count the number of words in the original article
    original_word_count = len(article.split())
    print(f"Original article word count: {original_word_count}")

    # Tokenize and encode the article
    inputs = tokenizer.encode(article, return_tensors='pt', max_length=1024, truncation=True)

    # Generate the summary with specified length constraints
    summary = ""
    while True:
        summary_ids = model.generate(
            inputs,
            num_beams=4,
            min_length=min_words,           # Set minimum tokens
            max_length=int(max_tokens),      # Set maximum tokens
            length_penalty=2.0,
            early_stopping=True
        )

        # Decode and calculate word count of the summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary_word_count = len(summary.split())

        # Check if the summary is within the specified range
        if min_words <= summary_word_count <= max_words:
            break

        # Reduce max_tokens slightly and retry if summary is too long
        max_tokens -= 5
        if max_tokens < min_words * 1.5:  # Stop if max_tokens is too low to avoid an infinite loop
            break

    # Print the generated summary word count
    print(f"Generated summary word count: {summary_word_count}")
    return summary

# Example usage
article = '''
Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.

Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.

TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.

'''
print("Original Text:")
print(article)
summary = summarize_article(article, max_words=20)
print("\nSummary:")
print(summary)
print(f"\nWord Count (Summary): {len(summary.split())}")


Original Text:

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will tr

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Original article word count: 421
Generated summary word count: 16

Summary:
Time Warner profits up 76% to $1.13bn for the three months to December. Firm is now

Word Count (Summary): 16


**ROUGE** Module

In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=56fac8b6b8da0f02436ef5ae0dbc722ba32f2ca50714ce1a6a5603d0a1a991fd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
def summarize_article(article):
    # Load BART model and tokenizer
    model_name = 'facebook/bart-large-cnn'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Count the number of words in the original article
    original_word_count = len(article.split())
    print(f"Original article word count: {original_word_count}")

    # Tokenize and encode the article
    inputs = tokenizer.encode(article, return_tensors='pt', max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs, num_beams=4, max_length=150, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Count the number of words in the summary
    summary_word_count = len(summary.split())
    print(f"Generated summary word count: {summary_word_count}")

    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(article, summary)

    # Print ROUGE scores
    print("ROUGE Scores:")
    for key, value in scores.items():
        print(f"{key}: {value}")

    return summary

# Example usage
article = '''
Ink helps drive democracy in Asia

The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting.

This new technology is causing both worries and guarded optimism among different sectors of the population. In an effort to live up to its reputation in the 1990s as "an island of democracy", the Kyrgyz President, Askar Akaev, pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections. The US government agreed to fund all expenses associated with this decision.

The Kyrgyz Republic is seen by many experts as backsliding from the high point it reached in the mid-1990s with a hastily pushed through referendum in 2003, reducing the legislative branch to one chamber with 75 deputies. The use of ink is only one part of a general effort to show commitment towards more open elections - the German Embassy, the Soros Foundation and the Kyrgyz government have all contributed to purchase transparent ballot boxes.

The actual technology behind the ink is not that complicated. The ink is sprayed on a person's left thumb. It dries and is not visible under normal light.

However, the presence of ultraviolet light (of the kind used to verify money) causes the ink to glow with a neon yellow light. At the entrance to each polling station, one election official will scan voter's fingers with UV lamp before allowing them to enter, and every voter will have his/her left thumb sprayed with ink before receiving the ballot. If the ink shows under the UV light the voter will not be allowed to enter the polling station. Likewise, any voter who refuses to be inked will not receive the ballot. These elections are assuming even greater significance because of two large factors - the upcoming parliamentary elections are a prelude to a potentially regime changing presidential election in the Autumn as well as the echo of recent elections in other former Soviet Republics, notably Ukraine and Georgia. The use of ink has been controversial - especially among groups perceived to be pro-government.

Widely circulated articles compared the use of ink to the rural practice of marking sheep - a still common metaphor in this primarily agricultural society.

The author of one such article began a petition drive against the use of the ink. The greatest part of the opposition to ink has often been sheer ignorance. Local newspapers have carried stories that the ink is harmful, radioactive or even that the ultraviolet readers may cause health problems. Others, such as the aggressively middle of the road, Coalition of Non-governmental Organizations, have lauded the move as an important step forward. This type of ink has been used in many elections in the world, in countries as varied as Serbia, South Africa, Indonesia and Turkey. The other common type of ink in elections is indelible visible ink - but as the elections in Afghanistan showed, improper use of this type of ink can cause additional problems. The use of "invisible" ink is not without its own problems. In most elections, numerous rumors have spread about it.

In Serbia, for example, both Christian and Islamic leaders assured their populations that its use was not contrary to religion. Other rumours are associated with how to remove the ink - various soft drinks, solvents and cleaning products are put forward. However, in reality, the ink is very effective at getting under the cuticle of the thumb and difficult to wash off. The ink stays on the finger for at least 72 hours and for up to a week. The use of ink and readers by itself is not a panacea for election ills. The passage of the inking law is, nevertheless, a clear step forward towards free and fair elections." The country's widely watched parliamentary elections are scheduled for 27 February.

David Mikosz works for the IFES, an international, non-profit organisation that supports the building of democratic societies.
'''


print("Original Text:")
print(article)
summary = summarize_article(article)
print("Summary:")
print(summary)

Original Text:

Ink helps drive democracy in Asia

The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting.

This new technology is causing both worries and guarded optimism among different sectors of the population. In an effort to live up to its reputation in the 1990s as "an island of democracy", the Kyrgyz President, Askar Akaev, pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections. The US government agreed to fund all expenses associated with this decision.

The Kyrgyz Republic is seen by many experts as backsliding from the high point it reached in the mid-1990s with a hastily pushed through referendum in 2003, reducing the legislative branch to one chamber with 75 deputies. The use of ink is only one part of a general effort to show commitment towards more open elections - the German

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Original article word count: 672
Generated summary word count: 49
ROUGE Scores:
rouge1: Score(precision=1.0, recall=0.07555555555555556, fmeasure=0.14049586776859505)
rouge2: Score(precision=0.96, recall=0.0712166172106825, fmeasure=0.13259668508287292)
rougeL: Score(precision=1.0, recall=0.07555555555555556, fmeasure=0.14049586776859505)
Summary:
The Kyrgyz Republic is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting. The use of ink has been controversial - especially among groups perceived to be pro-government. The country's widely watched parliamentary elections are scheduled for 27 February.
